# Отчёт по домашнему заданию

## Setup

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp /content/gdrive/My\ Drive/abbyy-nlp/train.jsonl /content
!cp /content/gdrive/My\ Drive/abbyy-nlp/dev.jsonl /content
# !cp /content/gdrive/My\ Drive/abbyy-nlp/wiki.en.zip /content
# !unzip wiki.en.zip

# !pip install gensim
# !pip install transformers
# !pip install nltk

## Load dataset

In [0]:
from pprint import pprint
import json

with open("train.jsonl", ) as reader:
    train_dataset = list(map(json.loads, reader))

with open("dev.jsonl", ) as reader:
    dev_dataset = list(map(json.loads, reader))


In [0]:
from random import randint

n_samples = 2
print("Примеры из обучающей выборки: \n")
_ = [pprint(train_dataset[randint(0, len(train_dataset)-1)]) for _ in range(n_samples)]
print("\n", "="*50, "\n")
print("Примеры из тестовой выборки: \n")
_ = [pprint(dev_dataset[randint(0, len(dev_dataset)-1)]) for _ in range(n_samples)]

Примеры из обучающей выборки: 

{'answer': False,
 'passage': 'At present, the euro is legal tender in 19 out of 28 European '
            'Union member states, in addition to 5 countries not part of the '
            'EU (Monaco, San Marino, Vatican City, Andorra and Montenegro). '
            'The Republic of Kosovo also uses the euro, but is only partially '
            'recognised as an independent state.',
 'question': 'the currency of all eu countries is the euro',
 'title': 'List of currencies in Europe'}
{'answer': True,
 'passage': 'The Czech Republic (/ˈtʃɛk -/ ( listen); Czech: Česká republika '
            '(ˈtʃɛskaː ˈrɛpublɪka) ( listen)), also known by its short-form '
            'name, Czechia (/ˈtʃɛkiə/ ( listen); Czech: Česko (ˈtʃɛsko) ( '
            'listen)), is a landlocked country in Central Europe bordered by '
            'Germany to the west, Austria to the south, Slovakia to the east '
            'and Poland to the northeast. The Czech Republic covers an are

# Часть 1. [1 балл] Эксплоративный анализ

## 1.1 Доля yes и no классов в корпусе

In [0]:
get_yes_ratio = lambda ds: sum(map(lambda x: x['answer'], ds)) / len(ds)

print("Обучающая выборка")
print("\t yes:", get_yes_ratio(train_dataset))
print(f"\t no:", 1 - get_yes_ratio(train_dataset))

print("Тестовая выборка")
print(f"\t yes:", get_yes_ratio(dev_dataset))
print(f"\t no:", 1 - get_yes_ratio(dev_dataset))

Обучающая выборка
	 yes: 0.6231038506417736
	 no: 0.37689614935822635
Тестовая выборка
	 yes: 0.6217125382262997
	 no: 0.3782874617737003


## 1.2-3 Средняя длина вопроса и параграфа

In [0]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()
get_word_count = lambda paragraph, field: len(tokenizer.tokenize(paragraph[field]))
get_avg_word_count = lambda ds, field: sum(map(lambda x: get_word_count(x, field), ds)) / len(ds)
get_avg_char_count = lambda ds, field: sum(map(lambda x: len(x[field]), ds)) / len(ds)

print("Средняя длина вопроса:")
print("\t в обучающей выборке:")
print("\t\t word:", get_avg_word_count(train_dataset, 'question'))
print("\t\t char:", get_avg_char_count(train_dataset, 'question'))

print("\t в тестовой выборке:")
print("\t\t word:", get_avg_word_count(dev_dataset, 'question'))
print("\t\t char:", get_avg_char_count(dev_dataset, 'question'))

print("Средняя длина параграфа:")
print("\t в обучающей выборке:")
print("\t\t word:", get_avg_word_count(train_dataset, 'passage'))
print("\t\t char:", get_avg_char_count(train_dataset, 'passage'))

print("\t в тестовой выборке:")
print("\t\t word:", get_avg_word_count(dev_dataset, 'passage'))
print("\t\t char:", get_avg_char_count(dev_dataset, 'passage'))

Средняя длина вопроса:
	 в обучающей выборке:
		 word: 8.831971995332555
		 char: 43.99193805028111
	 в тестовой выборке:
		 word: 8.726299694189603
		 char: 43.206422018348626
Средняя длина параграфа:
	 в обучающей выборке:
		 word: 109.20685265726105
		 char: 565.6130264134931
	 в тестовой выборке:
		 word: 108.27033639143731
		 char: 559.0522935779817


## Эвристики, по которым был собран датасет

Авторы статьи оставляли только те вопросы, которые начинались с одного из слова {“did”, “do”, “does”, “is”, “are”, “was”, “were”, “have”, “has”, “can”, “could”, “will”, “would”}. После этого фильтровались вопросы, которые при запросе в поиске гугл, не выдавали страницу с википедией среди первых 5 выдач.

In [0]:
from collections import Counter

starter_tokens = {"did", "do", "does", "is", "are", "was", "were", "have", "has", "can", "could", "will", "would"}
first_question_words = Counter([doc['question'].split()[0] for doc in train_dataset])

### 20 наиболее часто встречающихся первых токенов вопроса

In [0]:
pprint(first_question_words.most_common(20))

[('is', 4190),
 ('can', 1136),
 ('does', 952),
 ('are', 693),
 ('do', 664),
 ('did', 461),
 ('was', 335),
 ('has', 302),
 ('will', 181),
 ('the', 91),
 ('have', 70),
 ('in', 35),
 ('were', 25),
 ('if', 17),
 ('a', 16),
 ('what', 11),
 ('when', 10),
 ('could', 9),
 ('who', 5),
 ('where', 5)]


### Процент вопросов, начинающихся со слов "did", "do", "does", "is", "are", "was", "were", "have", "has", "can", "could", "will", "would"

In [0]:
sum([first_question_words[word] for word in starter_tokens]) / len(train_dataset)

0.9570382942611647

### Примеры вопросов, начинающихся не на данные слова

In [0]:
from random import choices
indices = [i for i, x in enumerate([doc['question'].split()[0] not in starter_tokens for doc in train_dataset]) if x]
subsample = choices(indices, k=3)
_ = [pprint(train_dataset[i]) for i in subsample]

{'answer': True,
 'passage': 'The Domain Name System (DNS) is a hierarchical decentralized '
            'naming system for computers, services, or other resources '
            'connected to the Internet or a private network. It associates '
            'various information with domain names assigned to each of the '
            'participating entities. Most prominently, it translates more '
            'readily memorized domain names to the numerical IP addresses '
            'needed for locating and identifying computer services and devices '
            'with the underlying network protocols. By providing a worldwide, '
            'distributed directory service, the Domain Name System is an '
            'essential component of the functionality on the Internet, that '
            'has been in use since 1985.',
 'question': 'the domain name system (dns) converts domain names to ip '
             'addresses',
 'title': 'Domain Name System'}
{'answer': True,
 'passage': 'In New Yor

# 2. Baseline

## 2.1 Dummy Classifier

In [0]:
train_questions = [doc['question'] for doc in train_dataset]
train_passages = [doc['passage'] for doc in train_dataset]
train_answers = [str(doc['answer']) for doc in train_dataset]

dev_questions = [doc['question'] for doc in dev_dataset]
dev_passages = [doc['passage'] for doc in dev_dataset]
dev_answers = [str(doc['answer']) for doc in dev_dataset]

### train

In [0]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

clf = DummyClassifier("most_frequent")
clf.fit(train_questions, train_answers)

DummyClassifier(constant=None, random_state=None, strategy='most_frequent')

### Evaluation

In [0]:
dev_preds = clf.predict(dev_questions)
print(classification_report(dev_answers, dev_preds))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00      1237
        True       0.62      1.00      0.77      2033

    accuracy                           0.62      3270
   macro avg       0.31      0.50      0.38      3270
weighted avg       0.39      0.62      0.48      3270



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 2.2 Fasttext

In [0]:
from gensim.models import FastText

model = FastText.load_fasttext_format("wiki.en.bin")

In [0]:
from typing import List
import numpy as np
from tqdm.auto import tqdm
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stopwords_ = set(stopwords.words('english'))

def get_text_vector(tokens: List[str], model):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    # return np.mean(vectors, axis=0)
    return np.hstack([np.mean(vectors, axis=0), np.max(vectors, axis=0), np.std(vectors, axis=0)])

def preprocess_text(text: str):
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    lemmas = map(lemmatizer.lemmatize, tokens)
    return [lemma for lemma in lemmas if lemma not in stopwords_]

def get_ds_vectors(dataset, model):
    vectors = []
    for doc in tqdm(dataset):
        text = doc['question'] + " " + doc["passage"]
        lemmas = preprocess_text(text)
        vectors.append(get_text_vector(lemmas, model))

    vectors = np.vstack(vectors)

    return vectors

train_vectors = get_ds_vectors(train_dataset, model)
dev_vectors = get_ds_vectors(dev_dataset, model)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=500, solver="liblinear")
clf.fit(train_vectors, train_answers)
dev_preds = clf.predict(dev_vectors)

print(classification_report(dev_answers, dev_preds))

              precision    recall  f1-score   support

       False       0.50      0.30      0.38      1237
        True       0.66      0.81      0.73      2033

    accuracy                           0.62      3270
   macro avg       0.58      0.56      0.55      3270
weighted avg       0.60      0.62      0.60      3270



# Часть 3 Эмбеддинги предложений

In [0]:
from transformers import BertModel, BertTokenizer
from transformers import pipeline
import torch 
import numpy as np
from tqdm.auto import tqdm

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

feature_extraction = pipeline("feature-extraction", model=bert_model, tokenizer=bert_tokenizer, device=0)

In [0]:
# np.asarray(feature_extraction([d['question'] for d in train_dataset[:10]], pad_to_max_length=True)).shape
def extract_features(texts):
    features = []
    for text in tqdm(texts):
        features.append(np.asarray(feature_extraction(text)).mean(axis=1)[0, :])
    return np.vstack(features)

train_question_features = extract_features([d['question'] for d in train_dataset])
train_passage_features = extract_features([d['passage'] for d in train_dataset])

dev_question_features = extract_features([d['question'] for d in dev_dataset])
dev_passage_features = extract_features([d['passage'] for d in dev_dataset])

In [0]:
# np.save("/content/gdrive/My Drive/abbyy-nlp/train_question_features.npy", train_question_features)
# np.save("/content/gdrive/My Drive/abbyy-nlp/train_passage_features.npy", train_passage_features)
# np.save("/content/gdrive/My Drive/abbyy-nlp/dev_question_features.npy", dev_question_features)
# np.save("/content/gdrive/My Drive/abbyy-nlp/dev_passage_features.npy", dev_passage_features)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

clf = LogisticRegression(max_iter=10000, solver="liblinear")
train_vectors = np.hstack((train_question_features, train_passage_features))
clf.fit(train_vectors, train_answers)
dev_vectors = np.hstack((dev_question_features, dev_passage_features))
dev_preds = clf.predict(dev_vectors)

print(classification_report(dev_answers, dev_preds))

# Часть 4. DrQA подобная архитектура

In [0]:
! git clone https://github.com/Muhamob/DrQA-1.git
! mv DrQA\-1/ drqa_light

Cloning into 'DrQA-1'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 395 (delta 182), reused 145 (delta 85), pack-reused 151
Receiving objects: 100% (395/395), 125.50 KiB | 11.41 MiB/s, done.
Resolving deltas: 100% (258/258), done.


In [0]:
# !cp -r drqa_light/glove /content/gdrive/My\ Drive/abbyy-nlp/
!mkdir drqa_light/glove
!cp -r /content/gdrive/My\ Drive/abbyy-nlp/glove/glove.840B.300d.txt drqa_light/glove/

In [0]:
!pip install -r drqa_light/requirements.txt

     |████████████████████████████████| 143kB 4.8MB/s 
     |████████████████████████████████| 3.4MB 12.7MB/s 
     |████████████████████████████████| 81kB 11.2MB/s 
     |████████████████████████████████| 931kB 44.9MB/s 
     |████████████████████████████████| 1.4MB 55.7MB/s 
     |████████████████████████████████| 184kB 57.9MB/s 
     |████████████████████████████████| 153kB 56.3MB/s 
     |████████████████████████████████| 614kB 49.7MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 389kB 54.3MB/s 
  Created wheel for msgpack-python: filename=msgpack_python-0.5.6-cp36-cp36m-linux_x86_64.whl size=299092 sha256=2c22901f04c0bad2c9b54f7238ad7bd9b6bdb39e8cb73f460205408e84dc02f6
  Stored in directory: /root/.cache/pip/wheels/d5/de/86/7fa56fda12511be47ea0808f3502bc879df4e63ab168ec0406
  Created wheel for spacy: filename=spacy-1.9.0-cp36-cp36m-linux_x86_64.whl size=7776139 sha256=d58a6bc454915d610b3673f3b8ce911847bc2a0c7fa398f78402bdc4a3cf85f

In [0]:
# %cd drqa_light
# !./download.sh
# %cd ..

In [0]:
%cd drqa_light
!git pull
!mkdir SQuAD/
!python prepro.py --trn_file ../train.jsonl --dev_file ../dev.jsonl --wv_file glove/glove.840B.300d.txt
%cd ..

/content/drqa_light
Already up to date.
05/05/2020 08:51:17 {'trn_file': '../train.jsonl', 'dev_file': '../dev.jsonl', 'wv_file': 'glove/glove.840B.300d.txt', 'wv_dim': 300, 'wv_cased': True, 'sort_all': False, 'sample_size': 0, 'threads': 2, 'batch_size': 64}
05/05/2020 08:51:17 start data preparing...
05/05/2020 08:51:17 json data flattened.


    Only loading the 'en' tokenizer.



    Only loading the 'en' tokenizer.

train: 100% 9427/9427 [00:11<00:00, 839.05it/s]
dev  : 100% 3270/3270 [00:03<00:00, 915.39it/s]
05/05/2020 08:51:32 drop 0 inconsistent samples.
05/05/2020 08:51:32 tokens generated
05/05/2020 08:52:05 glove vocab loaded.
05/05/2020 08:52:05 vocab coverage 57018/61545 | OOV occurrence 7483/1500604 (0.4987%)
05/05/2020 08:52:06 Vocabulary size: 57018
05/05/2020 08:52:06 Found 9 POS tags.
05/05/2020 08:52:06 Found 1 entity tags: ['']
05/05/2020 08:52:07 converted to ids.
05/05/2020 08:52:41 got embedding matrix.
05/05/2020 08:52:44 saved to disk.
/content


In [0]:
%cd drqa_light
!git pull
!rm -rf models/
!mkdir models/
!python train.py \
    --log_per_updates 1 \
    --doc_layers 2 \
    --question_layers 2 \
    --hidden_size 64 \
    --grad_clipping 3
%cd ..

[Errno 2] No such file or directory: 'drqa_light'
/content
fatal: not a git repository (or any of the parent directories): .git


KeyboardInterrupt: ignored